## Test Code for the cGAN

In [8]:
import torch
import pathlib
def celeb_label_transform(desired_attr):
    """
    Pass in a the names of all the attributes that you want
    """

    file = open('C:/Users/Paddy/CRT/Github/input/CelebA/list_attr_celeba.csv').read().split()
    attr_names = file[0].split(',')
    file = file[1:]
    
    def transform(idx):
        attr = torch.tensor([int(entry) for entry in file[idx].split(',')[1:]])
        mask = [attr_names[1:][i] in desired_attr for i in range(len(attr))]
        masked = attr[mask]
        return torch.relu(masked).float()
    return transform

file = open('C:/Users/Paddy/CRT/Github/input/CelebA/list_attr_celeba.csv').read().split()
attr_names = file[0].split(',')

bald_transform = celeb_label_transform("Bald")
bald_transform(50)


tensor([1.])

In [9]:
import pandas as pd

targ_dir = 'C:/Users/Paddy/CRT/Github/input/SEN12MS/'
filenames = list(pathlib.Path(targ_dir).glob("*/*/*.tif"))

# dictionary of lists
dict = {'name': filenames}
     
df = pd.DataFrame(filenames)
     
# saving the dataframe
df.to_csv('sen12_filenames.csv')

In [40]:
import torch

def sen12_label_transform(desired_season):
    """
    Pass in a the names of all the attributes that you want
    """

    file = open('C:/Users/Paddy/CRT/Github/input/SEN12MS/seasons_labeled_spring.csv').read().split()
    season_names = file[0].split(',')
    file = file[1:]
    
    def transform(idx):
        season = torch.tensor([int(entry) for entry in file[idx].split(',')[1:]])
        mask = [season_names[1:][i] in desired_season for i in range(len(season))]
        masked = season[mask]
        return torch.relu(masked).float()
    return transform

tensor([1.])

In [36]:
import os
import pandas as pd
import pathlib
from tqdm import tqdm

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/"
filenames = list(pathlib.Path(path).glob("*/*/*.tif"))

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/ROIs1158_spring"
# s1 = fall, s6 = spring

foldernames = list(pathlib.Path(path).glob("*/"))

seasons_original = pd.read_csv('C:/Users/Paddy/CRT/Github/input/SEN12MS/seasons_spring.csv')

seasons = []
filenames = []

for folder in foldernames:
    for file in list(pathlib.Path(folder).glob("*.tif")):
        season_ref = "_".join(os.path.basename(file).split("_")[0:4])
        season = seasons_original.loc[seasons_original["scene"] == season_ref]["true_season"]
        filenames.append(os.path.basename(file))
        seasons.append(season.item())

spring = [1 if x == 'spring' else -1 for x in seasons]
summer = [1 if x == 'summer' else -1 for x in seasons]
fall = [1 if x == 'fall' else -1 for x in seasons]
winter = [1 if x == 'winter' else -1 for x in seasons]

df = pd.DataFrame(zip(filenames, spring, summer, fall, winter), columns=['scene','spring', 'summer', 'fall', 'winter'])

print("Saving data...")
df.to_csv("seasons_labeled_spring.csv", index=False)
print("Saved.")
print(df.head())

Saving data...
Saved.
                           scene  spring  summer  fall  winter
0  ROIs1158_spring_s2_1_p100.tif      -1      -1     1      -1
1  ROIs1158_spring_s2_1_p101.tif      -1      -1     1      -1
2  ROIs1158_spring_s2_1_p102.tif      -1      -1     1      -1
3  ROIs1158_spring_s2_1_p103.tif      -1      -1     1      -1
4  ROIs1158_spring_s2_1_p104.tif      -1      -1     1      -1


In [2]:
import os
import pandas as pd
import pathlib
from tqdm import tqdm

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/"
filenames = list(pathlib.Path(path).glob("*/*/*.tif"))

print(filenames[0])

C:\Users\Paddy\CRT\Github\input\SEN12MS\ROIs1158_spring\s2_1\ROIs1158_spring_s2_1_p100.tif


In [5]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
from model import *
from transforms import *
from training_loop import *
from celeba_data import *
from sen12_data import *
from torch.utils.data import DataLoader
import os
import argparse
from torchinfo import summary
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

topn = 1
checkpoint_dir = os.path.join(os.path.dirname(os.getcwd()), "checkpoints")
name = 'sen12_cgan_test'
batch_size = 32
gen_steps = 1
disc_steps = 1
epochs = 1
img_size = 256
lr = 0.0002
beta = 0.5
desired_season = ['fall']
desired_attr = ['Young']
label_size = len(desired_season)
data_source = "C:/Users/Paddy/CRT/Github/input/SEN12MS"
#data_source = "C:/Users/Paddy/CRT/Github/input/sen12_overfit/ROIs1158_spring_s2_1_p30.tif"
source_labels = "C:/Users/Paddy/CRT/Github/input/SEN12MS/seasons_labeled_spring.csv"
#source_labels = "C:/Users/Paddy/CRT/Github/input/sen12_overfit/seasons_labeled_overfit.csv"
bands = "rgb" # or "rgb"
img_channels = 3

transform_sen = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Resize((img_size,img_size),antialias=False),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize([0.5 for _ in range(img_channels)],[0.5 for _ in range(img_channels)])
])
anntransform_celeb = celeb_label_transform(desired_attr)

imgtransform = BasicImageCropTransform(size = (img_size, img_size), scale = (1, 2))

dataset_celeba = CelebDS(imgtransform, anntransform_celeb)

first_celeb = dataset_celeba[0]

print("CelebA Input dimenstion:", first_celeb)


CelebA Input dimenstion: (tensor([[[ 1.0000,  0.8740,  0.9569,  ...,  0.9301,  0.8702,  1.0000],
         [ 0.8891,  1.0000,  0.8907,  ...,  0.9869,  0.8675,  0.8941],
         [ 1.0000,  1.0000,  0.9431,  ...,  0.9423,  1.0000,  0.7977],
         ...,
         [ 0.3281,  0.0857,  0.4427,  ...,  0.1894,  0.3279,  0.2113],
         [ 0.3590,  0.3182,  0.3662,  ...,  0.4533,  0.3124,  0.3469],
         [ 0.6486,  0.4645,  0.3175,  ...,  0.5201,  0.4995,  0.6231]],

        [[ 0.9102,  0.9289,  0.7557,  ...,  0.8395,  0.8781,  1.0000],
         [ 0.7561,  0.8683,  0.7075,  ...,  0.9468,  0.8658,  0.7826],
         [ 0.8935,  0.8591,  0.7188,  ...,  0.8613,  0.9031,  1.0000],
         ...,
         [-0.1999, -0.2934,  0.0103,  ..., -0.0490,  0.0831, -0.1797],
         [-0.3099,  0.0180,  0.0302,  ..., -0.0168,  0.1033, -0.1155],
         [-0.0581, -0.0994,  0.0322,  ...,  0.0101,  0.0698,  0.1899]],

        [[ 0.4857,  0.3754,  0.4326,  ...,  0.6677,  0.8623,  0.9273],
         [ 0.6518, 

In [16]:
print("Sen12 Input dimenstion:", first_sen12)
print("Max = ", torch.max(first_sen12))
print("Min = ", torch.min(first_sen12))
#print("Sen12MS Input dimenstion:", dataset_sen12[0][0])

Sen12 Input dimenstion: torch.Size([256, 256])
Max =  tensor(1.)
Min =  tensor(-1.)
